# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix, f1_score ,precision_score , accuracy_score,recall_score 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
conn = sqlite3.connect('database.db')
#engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM msgs_cat ', con = conn)

X = df['message'].values
Y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
   
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok,pos='v').lower().strip()
       # if clean_tok not in stop_words:
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    

    def tokenizee(self,text):
        url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        detected_urls = re.findall(url_regex, text)
        try:
            for url in detected_urls:
                text = text.replace(url, "urlplaceholder")
            text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
            tokens = word_tokenize(text)
        except:
            print('1failed\n\n')
        lemmatizer = WordNetLemmatizer()
        try:
            clean_tokens = []
            for tok in tokens:
                clean_tok = lemmatizer.lemmatize(tok).lower().strip()
                clean_tokens.append(clean_tok)
        except:
            print('2failed\n\n')
        return clean_tokens
    
    def starting_verb(self, text):
        
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            
            
            try:
                s=self.tokenizee(sentence)
                if len(s)>0:
                    pos_tags = nltk.pos_tag(s)
                    #print(pos_tags)
                    first_word, first_tag = pos_tags[0]
                    if first_tag in ['VB', 'VBP'] or first_word == 'rt':
                        return True
                    else:
                        return False
                else:
                    return False
            except: 
                try:
                    print(str(sentence)+'\n\n')
                    print(str(self.tokenizee(sentence))+'\n\n')
                    print('postags12  '+str(nltk.pos_tag(clean_tokens))+'\n\n')
                    print('postags22  '+ str(pos_tags)+'\n\n')
                    print('postags  '+ str(pos_tags[0])+'\n\n')
                except: print('non\n\n')
                return False
                

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb).fillna(False).values
        
        return np.array(X_tagged).reshape(-1,1).astype(float)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [16]:

pipeline2.fit(x_train,y_train)
y_pred = pipeline2.predict(x_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [60]:

def evaluate(y_test,y_pred,df):
    result = []
    for i in range(len(df.columns[4:])):
        f1score = f1_score(y_test[:,i],y_pred[:,i])
        precision = precision_score(y_test[:,i],y_pred[:,i])
        recall = recall_score(y_test[:,i],y_pred[:,i])
        col_name = df.columns[i+4]
        result.append([col_name,f1score,precision,recall])
    results = pd.DataFrame(result,columns=['col_name','f1score','precision','recall'])
    
    print(results)

In [61]:
evaluate(y_test,y_pred,df)

                  col_name   f1score  precision    recall
0                  related  0.871112   0.823385  0.924712
1                  request  0.490596   0.812987  0.351291
2                    offer  0.000000   0.000000  0.000000
3              aid_related  0.614415   0.754882  0.518022
4             medical_help  0.137500   0.733333  0.075862
5         medical_products  0.093960   0.736842  0.050179
6        search_and_rescue  0.079470   0.400000  0.044118
7                 security  0.000000   0.000000  0.000000
8                 military  0.146067   0.650000  0.082278
9              child_alone  0.000000   0.000000  0.000000
10                   water  0.480519   0.874016  0.331343
11                    food  0.414062   0.864130  0.272260
12                 shelter  0.320690   0.830357  0.198718
13                clothing  0.150000   0.600000  0.085714
14                   money  0.068966   1.000000  0.035714
15          missing_people  0.030769   0.500000  0.015873
16            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
df.columns[4:].values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object)

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline2.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [25]:
parameters = {'clf__estimator__criterion': ['gini'],
              'clf__estimator__n_estimators': [ 40],
              'features__text_pipeline__vect__ngram_range' : [(1,2)],
             'features__transformer_weights': [
            {'text_pipeline': 1, 'starting_verb': 0.5}]}


parameters2 = {'clf__estimator__criterion': ['gini','entropy'],
              'clf__estimator__n_estimators': [10, 20, 40],
              'features__text_pipeline__vect__ngram_range' : ((1,1),(1,2)),
             'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
             {'text_pipeline': 1, 'starting_verb': 1})}

cv = GridSearchCV(pipeline2, param_grid=parameters, scoring='f1_micro',n_jobs=-1,cv=3,verbose=5)
cv.fit(x_train, y_train)

cv.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.5975024015369836, total= 5.9min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.9min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.6035267677058722, total= 6.1min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 13.9min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=40, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.6078066382198803, total= 6.0min


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 20.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 20.8min finished


{'clf__estimator__criterion': 'gini',
 'clf__estimator__n_estimators': 40,
 'features__text_pipeline__vect__ngram_range': (1, 2),
 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 0.5}}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [62]:
y_pred2 = cv.predict(x_test)
evaluate(y_test,y_pred2,df)

                  col_name   f1score  precision    recall
0                  related  0.880219   0.809144  0.964982
1                  request  0.567752   0.872093  0.420875
2                    offer  0.000000   0.000000  0.000000
3              aid_related  0.665236   0.792839  0.573013
4             medical_help  0.105708   0.657895  0.057471
5         medical_products  0.131148   0.769231  0.071685
6        search_and_rescue  0.095890   0.700000  0.051471
7                 security  0.020408   0.500000  0.010417
8                 military  0.059524   0.500000  0.031646
9              child_alone  0.000000   0.000000  0.000000
10                   water  0.413636   0.866667  0.271642
11                    food  0.623632   0.863636  0.488014
12                 shelter  0.439873   0.847561  0.297009
13                clothing  0.258824   0.733333  0.157143
14                   money  0.132231   0.888889  0.071429
15          missing_people  0.000000   0.000000  0.000000
16            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [56]:
model2 = pipeline.fit(x_train, y_train)



In [57]:
y_pred = model2.predict(x_test)

In [63]:
evaluate(y_test,y_pred,df)

                  col_name   f1score  precision    recall
0                  related  0.871112   0.823385  0.924712
1                  request  0.490596   0.812987  0.351291
2                    offer  0.000000   0.000000  0.000000
3              aid_related  0.614415   0.754882  0.518022
4             medical_help  0.137500   0.733333  0.075862
5         medical_products  0.093960   0.736842  0.050179
6        search_and_rescue  0.079470   0.400000  0.044118
7                 security  0.000000   0.000000  0.000000
8                 military  0.146067   0.650000  0.082278
9              child_alone  0.000000   0.000000  0.000000
10                   water  0.480519   0.874016  0.331343
11                    food  0.414062   0.864130  0.272260
12                 shelter  0.320690   0.830357  0.198718
13                clothing  0.150000   0.600000  0.085714
14                   money  0.068966   1.000000  0.035714
15          missing_people  0.030769   0.500000  0.015873
16            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [26]:
def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)
        
import pickle
save_model(cv , 'classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.